In [1]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

TensorFlow version: 2.10.0
GPU is available


In [2]:
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from tensorflow.keras.applications import ResNet50
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from IPython.display import Image

In [3]:
img_height = 32
img_width = 32
batch_size = 500

# Load the training data
train_ds = tf.keras.utils.image_dataset_from_directory(
  "train",
  seed = 512,
  image_size = (img_height, img_width),
  batch_size = batch_size)

# Load the validation data
val_ds = tf.keras.utils.image_dataset_from_directory(
  "validation",
  seed = 512,
  image_size = (img_height, img_width),
  batch_size = batch_size)

print("Training Classes:")
class_names = train_ds.class_names
print(class_names)

print("Testing Classes:")
class_names = val_ds.class_names
print(class_names)

Found 80000 files belonging to 2 classes.
Found 20000 files belonging to 2 classes.
Training Classes:
['FAKE', 'REAL']
Testing Classes:
['FAKE', 'REAL']


## Using ResNet50 pretrained model

include_top=False: Removes the top (classification) layer of ResNet50, leaving only the convolutional layers. This allows the model to act as a feature extractor.

In [11]:
# Load the ResNet50 base model without the top layer and with ImageNet weights
ResNet_base_model = tf.keras.applications.ResNet50(
    include_top = False, 
    weights = 'imagenet', 
    input_shape = (img_height, img_width, 3),
    pooling = 'max'
)

# Set ResNet50 layers as trainable
ResNet_base_model.trainable = True

inputs = keras.Input(shape=(img_height, img_width, 3))

# Pass the input through the ResNet50 base model
x = ResNet_base_model(inputs, training=False)

# Add Batch Normalization
x = layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(x)

x = layers.Dense(
    256,
    activation="relu",
    kernel_regularizer=regularizers.l2(0.01),
    activity_regularizer=regularizers.l1(0.01),
    bias_regularizer=regularizers.l1(0.01)
)(x)

x = layers.Dropout(rate=0.4, seed=512)(x)
x = layers.Dense(64, activation="relu")(x)

outputs = layers.Dense(1, activation="sigmoid")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

model.compile(
    optimizer=keras.optimizers.Adamax(learning_rate=0.001),
    loss=keras.losses.BinaryCrossentropy(),
    metrics=['accuracy', keras.metrics.Precision(), keras.metrics.Recall()]
)

# Display model summary
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 batch_normalization_1 (Batc  (None, 2048)             8192      
 hNormalization)                                                 
                                                                 
 dense_3 (Dense)             (None, 256)               524544    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_4 (Dense)             (None, 64)                16448     
                                                           

In [12]:
print("Starting training with Transfer Learning using ResNet50...")
ResNet_model_history = model.fit(
    train_ds,
    validation_data = val_ds,
    epochs = 100,
    verbose = 1,
    callbacks = [early_stopping]
)
print("Transfer Learning training finished.")

Starting training with Transfer Learning using ResNet50...
Epoch 1/100
160/160 [==============================] - 34s 179ms/step - loss: 2.3834 - accuracy: 0.8934 - precision_1: 0.8918 - recall_1: 0.8954 - val_loss: 1.9111 - val_accuracy: 0.8791 - val_precision_1: 0.8115 - val_recall_1: 0.9877
Epoch 2/100
160/160 [==============================] - 26s 161ms/step - loss: 0.7803 - accuracy: 0.9384 - precision_1: 0.9399 - recall_1: 0.9368 - val_loss: 1.0971 - val_accuracy: 0.8213 - val_precision_1: 0.7377 - val_recall_1: 0.9972
Epoch 3/100
160/160 [==============================] - 27s 162ms/step - loss: 0.3809 - accuracy: 0.9471 - precision_1: 0.9482 - recall_1: 0.9459 - val_loss: 0.3550 - val_accuracy: 0.9463 - val_precision_1: 0.9274 - val_recall_1: 0.9684
Epoch 4/100
160/160 [==============================] - 27s 167ms/step - loss: 0.2331 - accuracy: 0.9554 - precision_1: 0.9555 - recall_1: 0.9554 - val_loss: 0.2889 - val_accuracy: 0.9258 - val_precision_1: 0.8781 - val_recall_1: 0.98

### Accuracy on test dataset

In [13]:
test_ds = tf.keras.utils.image_dataset_from_directory(
  "test",
  seed = 512,
  image_size = (img_height, img_width),
  batch_size = batch_size)

# Evaluate the model on the test dataset
val_loss, val_accuracy, val_precision, val_recall = model.evaluate(test_ds)

# Print the metrics
print(f"Val Loss: {val_loss:.4f}")
print(f"Val Accuracy: {val_accuracy:.4f}")
print(f"Val Precision: {val_precision:.4f}")
print(f"Val Recall: {val_recall:.4f}")

Found 20000 files belonging to 2 classes.
40/40 [==============================] - 4s 67ms/step - loss: 0.1685 - accuracy: 0.9538 - precision_1: 0.9523 - recall_1: 0.9553
Val Loss: 0.1685
Val Accuracy: 0.9538
Val Precision: 0.9523
Val Recall: 0.9553
